In [1]:
import pickle
import numpy as np

In [5]:
with open('./data/test.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [8]:
test_data=np.array(test_data)
print(test_data)
x=test_data[:,:-1]
y=test_data[:,1]
print(x)
print(y)



[[-2.14700000e+01 -5.02846044e+02]
 [-3.71100000e+01 -2.58147766e+03]
 [-6.40000000e+00 -1.28624000e+01]
 [ 3.61000000e+00  3.05555205e+00]
 [ 2.57700000e+01  8.43144144e+02]
 [-3.68500000e+01 -2.52776216e+03]
 [ 4.54200000e+01  4.64430568e+03]
 [ 9.72000000e+00  4.49297344e+01]
 [ 1.21600000e+01  8.78233728e+01]
 [ 3.21500000e+01  1.64155422e+03]
 [ 3.73700000e+01  2.58208779e+03]
 [ 3.30200000e+01  1.77898237e+03]
 [ 6.13000000e+00  1.17044818e+01]
 [-8.12000000e+00 -2.70068544e+01]
 [-4.63000000e+01 -5.00405315e+03]
 [ 3.81000000e+00  3.43689505e+00]
 [-2.39000000e+00  2.27062050e-01]
 [-5.01000000e+00 -5.73947705e+00]
 [-4.98100000e+01 -6.22714303e+03]
 [-1.44000000e+00  8.23628800e-01]
 [ 1.71000000e+00  1.17442855e+00]
 [-3.26900000e+01 -1.76673153e+03]
 [ 3.87600000e+01  2.88209592e+03]
 [-6.51000000e+00 -1.35772246e+01]
 [ 4.24000000e+00  4.40929920e+00]
 [-3.82100000e+01 -2.81715582e+03]
 [ 1.05800000e+01  5.78698276e+01]
 [-2.98200000e+01 -1.34233196e+03]
 [-4.88200000e+01 -5